# Web Scraping Task
For this task, you are required to extract information from school list.xlsx. In the document, you will fin 
a table with 499 schools’ names and unique identifier code (DANE code2) and other information of ea h
school as municipality, state, and one score assigned to each school. We need you to collect information f om
the Colombian Ministry of Education website3 using any web-scraping technique that you feel comfort ble
with. You should obtain a complete database with the information required to complete the 

Tip: If Python is your chosen programming language, you will need a driver so that Selenium can interact 
with your browser of choice.4
1. Load the packages that you need for this ta
2. Extract the information from school list.pdf and create a database with it.
3. You have to scrape the information from the Colombian Ministry of Education website: https:/
sineb.mineducacion.gov.co/bcol/app. You also need to set the webdriver that you are going  o
use ank.task.

In [ ]:
!pip install selenium
!pip install webdriver-manager

In [2]:
# Core scraping tools
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Driver manager (auto-downloads the right driver)
from webdriver_manager.chrome import ChromeDriverManager

# File and data handling
import pandas as pd
import time
import os
import re

# To suppress Selenium logs (optional)
import logging
logging.getLogger('selenium').setLevel(logging.CRITICAL)

4. Create an empty dataframe to store the collected information. You will collect 10 items for each school.

In [16]:
# Define the column names for the collected school data
columns = [
    "school_name",       # Nombre
    "dane_code",         # Código DANE
    "address",           # Dirección
    "phone",             # Teléfono
    "department",        # Departamento
    "municipality",      # Municipio
    "calendar",          # Calendario
    "sector",            # Sector
    "area",              # Zona EE
    "gender"             # Género
    # "school_day",      # Optional: Jornadas
    # "dean"            # Optional: Rector
]

# Create the empty DataFrame
school_data = pd.DataFrame(columns=columns)

5. For each school, you will need to perform the following steps:

   1. **Open the webpage**:  
      [https://sineb.mineducacion.gov.co/bcol/app](https://sineb.mineducacion.gov.co/bcol/app)

   2. **Enter the Sede Code** into the field labeled **"Código DUE Sede"**.

   3. **Click the “Consultar” button** to submit the search.

   4. **Click the school link** that appears on the next page (it will be labeled with the school’s name).

   5. On the final page, **extract the following information**:
      - `Nombre` (Name)  
      - `Código DANE` (School's code)  
      - `Dirección` (Address)  
      - `Teléfono` (Phone)  
      - `Departamento` (State)  
      - `Municipio` (Municipality)  
      - `Calendario` (Calendar type)  
      - `Sector` (Sector)  
      - `Zona EE` (Area)  
      - `Género` (Gender)  
      - `Jornadas` (School day type)  
     - `Rector` (Dean)
 
   - `Rector` (Dean)


In [30]:
# Correct path and file name (with dash)
excel_path = r"C:\Users\SEBASTIAN\Documents\tosend\schools-list.xlsx"

# Load the Excel file
school_df = pd.read_excel(excel_path)

# Show confirmation and preview
print("✅ File loaded. Number of schools:", len(school_df))
school_subset = school_df.head(499)
school_subset.head()

✅ File loaded. Number of schools: 499


,sede_code,municipality_code,municipality_name,state_code,state_name,score
0,219780001218,19780,SUAREZ,19,CAUCA,63.566487
1,227495000260,27495,NUQUI,27,CHOCO,77.982206
2,176109002802,76109,BUENAVENTURA,76,VALLE,65.193998
3,227205000954,27580,RIO IRO,27,CHOCO,51.244012
4,376109005797,76109,BUENAVENTURA,76,VALLE,79.458485


In [31]:
# Setup Selenium
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)
action = ActionChains(driver)

# Storage for results
scraped_rows = []

# Label mapping from table to column names
label_map = {
    "Nombre:": "school_name",
    "Código DANE:": "dane_code",
    "Dirección:": "address",
    "Teléfono:": "phone",
    "Departamento:": "department",
    "Municipio:": "municipality",
    "Calendario:": "calendar",
    "Sector:": "sector",
    "Zona EE:": "area",
    "Rector:": "dean"
}

# Scraping loop
for _, row in school_subset.iterrows():
    sede_code = str(row["sede_code"]).strip()
    print(f"\n🔎 Searching for: {sede_code}")

    try:
        # Step 1: Open advanced search page
        driver.get("https://sineb.mineducacion.gov.co/bcol/app?service=page/BuscandoColegio")

        # Step 2: Input sede code
        searchTextbox = wait.until(EC.presence_of_element_located((By.NAME, "codigoSede")))
        action.move_to_element(searchTextbox).click().send_keys(sede_code).perform()

        # Step 3: Click "Consultar"
        consultar_btn = wait.until(EC.element_to_be_clickable((By.NAME, "consultar1")))
        action.move_to_element(consultar_btn).click().perform()

        # Step 4: Click on the school link
        school_link = wait.until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'linkNombre')]"))
        )
        school_url = school_link.get_property("href")
        print("✅ Navigating to:", school_url)
        driver.get(school_url)

        # Step 5: Extract values from info table
        data = {v: "" for v in label_map.values()}
        data["sede_code"] = sede_code

        info_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@width='100%']")))
        rows = info_table.find_elements(By.TAG_NAME, "tr")

        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            for i in range(len(cells)):
                label_el = cells[i].find_elements(By.TAG_NAME, "label")
                if label_el:
                    label = label_el[0].text.strip()
                    value_index = i + 1
                    if value_index < len(cells):
                        value = cells[value_index].text.strip()
                        if label in label_map:
                            data[label_map[label]] = value

        scraped_rows.append(data)
        print(f"✅ Scraped: {data.get('school_name', 'N/A')}")

    except Exception as e:
        print(f"❌ Error with sede_code {sede_code}: {e}")
        continue

# Cleanup
driver.quit()

# Merge and save output
scraped_df = pd.DataFrame(scraped_rows)
merged_df = school_df.merge(scraped_df, on="sede_code", how="left")
merged_df.to_csv("merged_school_data.csv", index=False)
print("📁 Data saved to: merged_school_data.csv")



🔎 Searching for: 219780001218
✅ Navigating to: https://sineb.mineducacion.gov.co/bcol/app;jsessionid=28AAD53E3E10F207C42FCAAFD0773B4D?service=direct/1/ResultadoBusqueda/linkNombre&sp=l3106
✅ Scraped: INSTITUCION EDUCATIVA ASNAZU

🔎 Searching for: 227495000260
✅ Navigating to: https://sineb.mineducacion.gov.co/bcol/app?service=direct/1/ResultadoBusqueda/linkNombre&sp=l8188
✅ Scraped: CE PASCUAL SANTANDER

🔎 Searching for: 176109002802
✅ Navigating to: https://sineb.mineducacion.gov.co/bcol/app?service=direct/1/ResultadoBusqueda/linkNombre&sp=l95048
✅ Scraped: INSTITUCION EDUCATIVA DIOCESANA JESUS ADOLESCENTE

🔎 Searching for: 227205000954
✅ Navigating to: https://sineb.mineducacion.gov.co/bcol/app?service=direct/1/ResultadoBusqueda/linkNombre&sp=l8178
✅ Scraped: IE JESUS ANTONIO RIVAS

🔎 Searching for: 376109005797
✅ Navigating to: https://sineb.mineducacion.gov.co/bcol/app?service=direct/1/ResultadoBusqueda/linkNombre&sp=l95016
✅ Scraped: PATRICIO SYMES

🔎 Searching for: 152473000517


In [34]:
merged_df.head(499)
os.getcwd()

'C:\\Users\\SEBASTIAN\\Webscrapping RA Task CUNEF'